# Uplift models и немного boosting



In [1]:
from All_Models import All_Models
from Models_for_Predictions import Models_for_Predictions
import numpy as np

Указываем путь к данным. В папке должны быть три датасета: features_trai.csv, features_val.csv, features_test.csv.
Указываем столбцы, которое будут удаляться при активации моделей. При иницилизации класса, столбцы для удаления в 
тестовом датасете сформируются сами, но если во время работы добавяться новые столбцы, которые нужно удалять, то нужно
добавить их в self.cols_to_drop и self.drop_columns_test

In [2]:
path = 'D:/ML_Sets/data/'
colsd = ['target', 'client_id', 'treatment_flg']
uplift = Models_for_Predictions(cols_to_drop = colsd, path_to_files = path, mode='test')

In [3]:
#Запускаем функцию классифицирующую клиента по 4 классам, в зависимости от  treatment_flg и target
uplift.create_classes()

In [4]:
#Меняем параметры в lgb модели, т.к. изначально стоят binary и auc
uplift.lgb_params['objective'] = 'regression'
uplift.lgb_params['metric'] = 'rmse'

In [5]:
#Запускаем модель, указывая нашу цель
pred_class_train, pred_class_test = uplift.lgb_stacking('classes')

In [6]:
#Добавляем полученые предсказания к нашим датасетам
uplift.train['pred_class'] = pred_class_train
uplift.test['pred_class'] = pred_class_test

In [7]:
#Запускаем класс с бинарной классификацией и фильтром по treatment_flg
colsd = ['target', 'client_id', 'treatment_flg', 'classes']
uplift1 = All_Models(cols_to_drop = colsd, mode='test')

In [8]:
#Передаем данные в новый класс
uplift1.train = uplift.train
uplift1.test = uplift.test

In [9]:
#Запускаем модель lgb
predict_control, predict_treatment, predict_train_0, predict_train_1, uplift_predict = uplift1.start_training_lgb('target')

In [10]:
#Добавляем полученные предсказания к данным
uplift1.train['lgb_0_auc_1'] = predict_train_0
uplift1.train['lgb_1_auc_1'] = predict_train_1
uplift1.test['lgb_0_auc_1'] = np.asarray(predict_control).mean(axis=0)
uplift1.test['lgb_1_auc_1'] = np.asarray(predict_treatment).mean(axis=0)

In [11]:
#Меняем параметры модели lgb
uplift1.lgb_params['objective'] = 'binary'
uplift1.lgb_params['metric'] = 'auc'

In [12]:
#Запускаем модель lgb
predict_control, predict_treatment, predict_train_0, predict_train_1, uplift_predict = tfd.start_training_lgb('target')

In [13]:
#Добавляем полученные предсказания к данным
uplift1.train['lgb_0_rmse_1'] = predict_train_0
uplift1.train['lgb_1_rmse_1'] = predict_train_1
uplift1.test['lgb_0_rmse_1'] = np.asarray(predict_control).mean(axis=0)
uplift1.test['lgb_1_rmse_1'] = np.asarray(predict_treatment).mean(axis=0)

In [14]:
#Запускаем модель cat
predict_control, predict_treatment, predict_train_0, predict_train_1, uplift_predict = uplift1.start_training_cat('target')

In [15]:
#Добавляем полученные предсказания к данным
uplift1.train['cat_0_auc_1'] = predict_train_0
uplift1.train['cat_1_auc_1'] = predict_train_1
uplift1.test['cat_0_auc_1'] = np.asarray(predict_control).mean(axis=0)
uplift1.test['cat_1_auc_1'] = np.asarray(predict_treatment).mean(axis=0)

Два самых влиятельных признака transactions и avg_delay_beetwen_transc, поэтому попробовал создать новые признаки на их основе и полученных предсказаниях. Эти же признаки удалить, чтобы модели не зацикливались на них.

In [16]:
#Составляем лист с полученными предсказаниями
col_lists =['pred_class', 'lgb_0_auc_1', 'lgb_1_auc_1','lgb_0_rmse_1', 'lgb_1_rmse_1', 'cat_0_auc_1', 'cat_1_auc_1']

In [17]:
#Генерируем новые признаки
def new_features():
    for col in col_lists:
        uplift1.train[col+'*tran'] = uplift1.train[col]*uplift1.train['transactions']*0.1
        uplift1.test[col+'*tran'] = uplift1.test[col]*uplift1.test['transactions']*0.1
        uplift1.train[col+'/age'] = uplift1.train[col]/uplift1.train['age']
        uplift1.test[col+'/age'] = uplift1.test[col]/uplift1.test['age']
        uplift1.train[col+'/age*avg_tran_d'] = uplift1.train[col]/uplift1.train['age']*uplift1.train['avg_delay_beetwen_transc']
        uplift1.test[col+'/age*avg_tran_d'] = uplift1.test[col]/uplift1.test['age']*uplift1.test['avg_delay_beetwen_transc']
        
new_features()

In [18]:
#Обновляем листы столбцов для удаления. 
cols_to_drop = ['treatment_flg', 'client_id', 'target', 'classes', 'transactions', 'avg_delay_beetwen_transc',]
uplift1.cols_to_drop = cols_to_drop
uplift1.drop_columns_test = ['client_id','transactions', 'avg_delay_beetwen_transc',]

In [19]:
#Запускаем модель lgb
predict_control, predict_treatment, predict_train_0, predict_train_1, uplift_predict = uplift1.start_training_lgb('target')

In [20]:
#Добавляем полученные данные
uplift1.train['lgb_0_rmse_2'] = predict_train_0
uplift1.train['lgb_1_rmse_2'] = predict_train_1
uplift1.test['lgb_0_rmse_2'] = np.asarray(predict_control).mean(axis=0)
uplift1.test['lgb_1_rmse_2'] = np.asarray(predict_treatment).mean(axis=0)

In [21]:
col_lists =['pred_class', 'lgb_0_auc_1', 'lgb_1_auc_1','lgb_0_rmse_1', 'lgb_1_rmse_1', 'cat_0_auc_1', 'cat_1_auc_1',\
           'lgb_0_rmse_2', 'lgb_1_rmse_2']

In [22]:
#Генерируем новые признаки
new_features()
uplift1.train['mean'] = uplift1.train.iloc[:, -36:].mean(axis=1)
uplift1.train['sum'] = uplift1.train.iloc[:, -36:].sum(axis=1)
uplift1.test['mean'] = uplift1.test.iloc[:, -36:].mean(axis=1)
uplift1.test['sum'] = uplift1.test.iloc[:, -36:].sum(axis=1)

In [23]:
#Запускаем модель cat
predict_control, predict_treatment, predict_train_0, predict_train_1, uplift_predict = uplift1.start_training_cat('target')

In [24]:
#Добавляем полученные данные
uplift1.train['cat_0_auc_2'] = predict_train_0
uplift1.train['cat_1_auc_2'] = predict_train_1
uplift1.test['cat_0_auc_2'] = np.asarray(predict_control).mean(axis=0)
uplift1.test['cat_1_auc_2'] = np.asarray(predict_treatment).mean(axis=0)

In [25]:
#Генерируем новые признаки
new_features()
uplift1.train['mean'] = uplift1.train.iloc[:, -39:].drop(['mean'], axis=1).mean(axis=1)
uplift1.train['sum'] = uplift1.train.iloc[:, -39:].drop(['mean'], axis=1).sum(axis=1)
uplift1.test['mean'] = uplift1.test.iloc[:, -39:].drop(['mean'], axis=1).mean(axis=1)
uplift1.test['sum'] = uplift1.test.iloc[:, -39:].drop(['mean'], axis=1).sum(axis=1)

In [26]:
#Обновляем листы столбцов для удаления, добавляя 
cols_to_drop = ['treatment_flg', 'client_id', 'target', 'classes', 'transactions', 'avg_delay_beetwen_transc',\
               'pred_class*tran',] 
uplift1.cols_to_drop = cols_to_drop
uplift1.drop_columns_test = ['client_id','transactions', 'avg_delay_beetwen_transc', 'pred_class*tran']

In [27]:
#Запускаем финальную модель lgb
predict_control, predict_treatment, predict_train_0, predict_train_1, uplift_predict = uplift1.start_training_lgb('target')

In [28]:
#Сохраняем предсказания
uplift_submission = pd.DataFrame((uplift1.test['client_id'], uplift_predict)).T
uplift_submission.columns = ['client_id', 'uplift']
uplift_submission.to_csv('D:/ML_Sets/uplift_submission.csv', index=False)

# P.S. Идеи и примечания

1. Очень сильно колебался uplift_score в зависимости от гиперпараметров. Даже округлив базовые до сотых, которые получены из hyperopt (если что они были подогнаны совсем под другой датасет. Взял их за отправную точку просто) uplift score на val set падал на 0.003.


2. hyperopt у меня не очень хотел работать с этими данными. Хотя я пробовал не долго, может стоит подождать 200+ evals. Плюс, думаю стоит попробовать его запускать именно под оптимизацию uplift score.


3. Совсем модели не работают на цель 'treatment_flg'.


4. В режиме low_memory отбрасываются признаки с товаров (level 1-4). Если их добавить, то может score улучшится.


5. Была идея предсказывать сумму последней покупки. Гипотеза в том, что если клиент купил меньше, чем предсказывает модель, то какие-то товары он не нашел по скидке и стоит ему ее предложить.


6. Найти и выбросить коррелирующие признаки.


7. Побольше признаков сгенерировать из предсказаний. Например, cat_0 - lgb_1, lgb_0+cat_0 и т.д. 


8. Получше проработать признак transcations(количество покупок). Например, выбрасывая pred_class*tran из признаков score растет. Можно выкинуть еще часть признаков умноженных на transcations и score держится более менее, но если выкинуть все, то он падает ниже 0.05 на val set.


9. Добавить цель target-treatment_flg, т.е. будет три класса: 1,0,-1.
